<a href="https://colab.research.google.com/github/tiagosilveiraa/portifoliods/blob/main/Recomenda%C3%A7%C3%A3o_de_Jogadores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [ ]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

#1º Coleta de dados



In [ ]:
#Competicoes para extracao de dados
LEAGUES = [{'id': 17, 'season': 37036 }, {'id':325, 'season': 36166}]
#Posicoes
POSITIONS = ['F']# Todas ['G', 'D', 'M', 'F']

In [ ]:
def make_url(competition, season, position, offset):
  return f"https://api.sofascore.com/api/v1/unique-tournament/{competition}/season/{season}/statistics?offset={offset}&order=-rating&accumulation=total&filters=position.in.{position}&fields=%2CbigChancesMissed%2CsuccessfulDribbles%2CsuccessfulDribblesPercentage%2CtotalShots%2CshotsOnTarget%2CshotsOffTarget%2CblockedShots%2CgoalConversionPercentage%2CpenaltiesTaken%2CpenaltyGoals%2CpenaltyWon%2CshotFromSetPiece%2CfreeKickGoal%2CgoalsFromInsideTheBox%2CgoalsFromOutsideTheBox%2CheadedGoals%2CleftFootGoals%2CrightFootGoal%2CshitWoodwork%2Coffsides%2CpenaltyConversion%2CsetPieceConversion%2Crating%2Cinterceptions%2CpenaltyConceded%2Cclearances%2CerrorLeadToGoal%2CerrorLeadToShot%2CownGoals%2CdribbledPast%2CcleanSheet%2C rating %2CbigChancesCreated%2Cassists %2CaccuratePasses%2CinaccuratePasses%2CtotalPasses%2CaccuratePassesPercentage %2CaccurateOwnHalfPasses%2CaccurateOppositionHalfPasses%2CaccurateFinalThirdPasses%2CkeyPasses%2CaccurateCrosses%2CaccurateCrossesPercentage%2CaccurateLongBalls%2CaccurateLongBallsPercentage%2CpassToAssist%2Crating %2CcleanSheet%2CpenaltyFaced%2CpenaltySave%2CsavedShotsFromInsideTheBox%2CsavedShotsFromOutsideTheBox%2CgoalsConcededInsideTheBox%2CgoalsConcededOutsideTheBox%2Cpunches%2CsuccessfulRunsOut%2ChighClaims%2CcrossesNotClaimed%2Crating %2CyellowCards%2CredCards%2CgroundDuelsWon%2CgroundDuelsWonPercentage%2CaerialDuelsWon%2CaerialDuelsWonPercentage%2CtotalDuelsWon%2CtotalDuelsWonPercentage%2CminutesPlayed%2CwasFouled%2Cfouls%2Cdispossessed%2CpossessionLost%2Cappearances%2CmatchesStarted%2Crating &limit=20"

In [ ]:
def get_stats(competitions):
  stats = pd.DataFrame()  
  for c in competitions:
    for p in POSITIONS:
      i = 0
      while True:  
        response = requests.get(make_url(c['id'], c['season'], p, i))
        #Já percoreu todos os players então paro
        if response.json()['results'] == []:        
          break;

        response = response.json()['results']
        response = pd.json_normalize(response, max_level=6)
        response['position'] = p        
        stats = pd.concat([stats, response])    
        i = i + 20 
  return stats

In [ ]:
players = get_stats(LEAGUES)

#2º Tratamento dos dados


In [ ]:
#colocando todas as colunas em minusculo
players = players.rename(str.lower,axis='columns')    

In [ ]:
#Só quero players que estiveram em alguma partida na temporada
players = players[players['appearances'] >0]

In [ ]:
#Alterando o indice para o player.id
players.set_index('player.id', inplace = True)

#3º Análise exploratória



#4° Geração do Modelo

In [ ]:
numeric_stats = players.drop([
    'team.name',
'player.name',
'player.slug',
'player.usercount',
'team.name',
'team.slug',
'team.shortname',
'team.sport.name',
'team.sport.slug',
'team.sport.id',
'team.usercount',
'team.type',
'team.id',
'team.teamcolors.primary',
'team.teamcolors.secondary',
'team.teamcolors.text',
'position',
'rating'
], axis='columns')

In [ ]:
numeric_stats.columns

In [ ]:
from scipy.sparse import csr_matrix
stats_sparse = csr_matrix(numeric_stats)

In [ ]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute', n_neighbors=5)
model.fit(stats_sparse)

NearestNeighbors(algorithm='brute')

In [ ]:
distance, similar_players_index = model.kneighbors(numeric_stats.loc[840219].values.reshape(1,-1))
similar_players = pd.DataFrame()
similar_players = players.loc[players.index[similar_players_index[0]]]
similar_players['distances'] = distance[0]

In [ ]:
similar_players

,bigchancesmissed,successfuldribbles,successfuldribblespercentage,totalshots,shotsontarget,shotsofftarget,blockedshots,goalconversionpercentage,penaltiestaken,penaltygoals,penaltywon,shotfromsetpiece,freekickgoal,goalsfrominsidethebox,goalsfromoutsidethebox,headedgoals,leftfootgoals,offsides,penaltyconversion,setpiececonversion,rating,interceptions,penaltyconceded,clearances,errorleadtogoal,errorleadtoshot,owngoals,dribbledpast,cleansheet,bigchancescreated,accuratepasses,inaccuratepasses,totalpasses,accurateownhalfpasses,accurateoppositionhalfpasses,accuratefinalthirdpasses,keypasses,accuratecrosses,accuratecrossespercentage,accuratelongballs,accuratelongballspercentage,passtoassist,penaltyfaced,penaltysave,savedshotsfrominsidethebox,savedshotsfromoutsidethebox,goalsconcededinsidethebox,goalsconcededoutsidethebox,punches,successfulrunsout,highclaims,crossesnotclaimed,yellowcards,redcards,groundduelswon,groundduelswonpercentage,aerialduelswon,aerialduelswonpercentage,totalduelswon,totalduelswonpercentage,minutesplayed,wasfouled,fouls,dispossessed,possessionlost,appearances,matchesstarted,player.name,player.slug,player.usercount,team.name,team.slug,team.shortname,team.sport.name,team.sport.slug,team.sport.id,team.usercount,team.type,team.id,team.teamcolors.primary,team.teamcolors.secondary,team.teamcolors.text,position,distances
player.id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
840219,7,15,42.86,42,23,16,3,16.67,0,0,0,0,0,7,0,2,0,4,0.0,0.0,6.87,6,0,5,0,0,0,9,1,4,211,65,276,63,148,89,24,0,0.00,3,60.00,0,0,0,0,0,12,3,0,0,0,0,3,0,52,41.27,17,32.69,69,38.76,1414,21,22,28,173,24,15,Pedro,pedro,8231,Flamengo,flamengo,Flamengo,Football,football,1,0,0,5981,#52b030,#52b030,#ffffff,F,0.000000
361352,4,16,38.10,47,14,17,16,4.26,0,0,1,0,0,1,1,0,1,2,0.0,0.0,6.71,3,0,6,0,0,0,6,2,1,227,76,303,86,141,97,13,0,0.00,9,60.00,0,0,0,0,0,29,2,0,0,0,0,1,0,32,35.56,13,36.11,45,35.71,1414,6,13,13,179,23,17,Adam Armstrong,adam-armstrong,1135,Southampton,southampton,Southampton,Football,football,1,0,0,45,#52b030,#52b030,#ffffff,F,65.994460
870333,5,13,65.00,37,11,16,10,2.70,0,0,1,2,0,1,0,0,1,13,0.0,0.0,6.71,7,0,11,0,0,0,11,0,2,209,80,289,45,168,106,21,4,17.39,7,31.82,0,0,0,0,0,12,2,0,0,0,0,0,0,56,51.85,17,26.98,73,42.69,1340,27,20,16,168,31,12,Rodolfo,rodolfo,105,América Mineiro,america-mineiro,América Mineiro,Football,football,1,0,0,1973,#52b030,#52b030,#ffffff,F,97.866493
33902,12,10,47.62,43,22,17,4,13.95,0,0,1,0,0,6,0,4,0,4,0.0,0.0,6.79,5,0,18,0,0,0,7,3,1,212,61,273,30,182,134,18,0,0.00,2,66.67,0,0,0,0,0,10,6,0,0,0,0,0,0,37,41.11,47,47.96,84,44.68,1477,13,17,22,150,25,15,Danny Welbeck,danny-welbeck,1456,Brighton & Hove Albion,brighton-hove-albion,Brighton & Hove Albion,Football,football,1,0,0,30,#52b030,#52b030,#ffffff,F,104.493013
26989,4,8,53.33,41,11,20,10,4.88,0,0,0,0,0,2,0,0,0,17,0.0,0.0,6.73,4,0,12,0,0,0,10,3,4,181,121,302,51,133,81,12,3,42.86,4,33.33,0,0,0,0,0,13,2,0,0,0,0,3,0,44,42.31,55,44.72,99,43.61,1378,16,31,13,179,29,13,Jay Rodriguez,jay-rodriguez,376,Burnley,burnley,Burnley,Football,football,1,0,0,6,#52b030,#52b030,#ffffff,F,113.539502
